# Sierpinski Triangle Generator
This notebook uses Python's `turtle` and `numpy` to generate a Sierpinski Triangle using a recursive mid-point approach.

In [ ]:
import turtle
import time
import numpy as np
from copy import deepcopy

def sierpinski_recursive(size, initial, counter, tri, mid_points_queue, edge_coords):
    """
    Recursively draws the Sierpinski triangle by calculating midpoints of previous edges.
    """
    try:
        remove_list = []

        for pos in mid_points_queue:
            if not initial:
                tri.penup()
            
            tri.goto(pos[0], pos[1])
            tri.setheading(0)

            if counter == 1:
                edge_coords.append(np.round(tri.position(), 2))

            # Handle orientation based on point labels
            if len(pos) == 3:
                if pos[-1] == "=":
                    tri.setheading(300)
                    edge_coords.append(np.round(tri.position(), 2))
                elif pos[-1] == "-":
                    tri.setheading(180)
                    tri.left(60)
                    tri.forward(size)
                    tri.setheading(300)
                    edge_coords.append(np.round(tri.position(), 2))
                elif pos[-1] == "+":
                    tri.setheading(0)
                    tri.right(60)
                    tri.forward(size)
                    tri.setheading(300)
                    edge_coords.append(np.round(tri.position(), 2))

            tri.pendown()
            for i in range(3):
                tri.left(120)
                tri.forward(size)
                if len(pos) == 3 and i < 2:
                    edge_coords.append(np.round(tri.position(), 2))

            if mid_points_queue[-1] == pos and not initial:
                tri.penup()
                tri.goto(mid_points_queue[0][0], mid_points_queue[0][1])

            remove_list.append(pos)

        initial = False
        for val in remove_list:
            mid_points_queue.remove(val)

        return_edge = deepcopy(edge_coords[0:len(edge_coords)-1:3])
        return_counter = 0

        for i in range(3**counter):
            if (i+1) % 3 == 0:
                second_edge_coord = return_edge[return_counter]
                result = np.add(edge_coords[0], second_edge_coord)
                return_counter += 1
            else:
                second_edge_coord = edge_coords[1]
                result = np.add(edge_coords[0], second_edge_coord)

            mid_point = tuple(np.round(result / 2, 2))
            grad_mid_point = list(mid_point)
            
            if edge_coords[0][0] < second_edge_coord[0] and edge_coords[0][1] < second_edge_coord[1]:
                grad_mid_point.append("+")
            elif edge_coords[0][1] == second_edge_coord[1]:
                grad_mid_point.append("=")
            else:
                grad_mid_point.append("-")

            mid_points_queue.append(tuple(grad_mid_point))
            del edge_coords[0]

        turtle.update()

        if size < 25:
            print("Fractal generation complete.")
            return

        sierpinski_recursive(size / 2, False, counter + 1, tri, mid_points_queue, edge_coords)
    
    except turtle.Terminator:
        print("Animation stopped: Window closed.")
    except Exception as e:
        print(f"An error occurred: {e}")

def main():
    # Setup Screen
    screen = turtle.Screen()
    screen.clearscreen()
    screen.mode('world')
    turtle.tracer(0, 0) # Turn off animation for instant drawing if needed
    
    tri = turtle.Turtle()
    tri.speed(0)
    tri.hideturtle()
    
    # Parameters
    size = 400
    edge_coords = []
    mid_points_queue = []
    positions = [(0, 0)]

    # Calculate centered positions
    temp_tri = turtle.Turtle()
    temp_tri.hideturtle()
    temp_tri.penup()
    for _ in range(3):
        temp_tri.forward(size)
        temp_tri.left(120)
        positions.append(temp_tri.position())

    cx = sum(p[0] for p in positions) / len(positions)
    cy = sum(p[1] for p in positions) / len(positions)
    offset = (-cx, -cy)
    centered_positions = [(x + offset[0], y + offset[1]) for x, y in positions]

    # Draw the main containing triangle
    tri.penup()
    tri.goto(centered_positions[0])
    tri.pendown()
    for point in centered_positions[1:] + [centered_positions[0]]:
        tri.goto(point)

    # Initialize mid-points queue
    for i in range(3):
        p1 = np.array(centered_positions[i])
        p2 = np.array(centered_positions[(i + 1) % 3])
        midpoint = tuple(np.round((p1 + p2) / 2, 2))
        mid_points_queue.append((midpoint[0], midpoint[1], "="))

    # Start recursion
    sierpinski_recursive(size / 2, True, 1, tri, mid_points_queue, edge_coords)
    
    turtle.update()
    print("Done. Click the X to close.")
    turtle.mainloop()

if __name__ == "__main__":
    main()